In [1]:
ls

acl16.ipynb      image2.pdf             saxpy.pyc
classification/  image3.pdf             time-series-Clustering-shuo15.ipynb
clustering/      imagetot.pdf           toneData/
datasets/        NAACL16-tsclust.ipynb  toneSub/
dendrogram.png   README.md
image1.pdf       saxpy.py


In [2]:
% matplotlib inline

# non-OOP demo

In [4]:
from sklearn.metrics import classification_report
import matplotlib.pylab as plt
import numpy as np

def DTWDistance(s1, s2,w):
    DTW={}
    
    w = max(w, abs(len(s1)-len(s2)))
    
    for i in range(-1,len(s1)):
        for j in range(-1,len(s2)):
            DTW[(i, j)] = float('inf')
    DTW[(-1, -1)] = 0
  
    for i in range(len(s1)):
        for j in range(max(0, i-w), min(len(s2), i+w)):
            dist= (s1[i]-s2[j])**2
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])
		
    return sqrt(DTW[len(s1)-1, len(s2)-1])


def LB_Keogh(s1,s2,r):
    LB_sum=0
    for ind,i in enumerate(s1):
        
        lower_bound=min(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
        upper_bound=max(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
        
        if i>upper_bound:
            LB_sum=LB_sum+(i-upper_bound)**2
        elif i<lower_bound:
            LB_sum=LB_sum+(i-lower_bound)**2
    
    return sqrt(LB_sum)


from math import sqrt

def euclid_dist(t1,t2):
    return sqrt(sum((t1-t2)**2))


#see tutorial
from sklearn.metrics import classification_report

#knn with DTW using LB-Keogh
def knn(train,test,w):
    preds=[]
    #index, line value
    for ind,i in enumerate(test):
        min_dist=float('inf')
        closest_seq=[]
        #print ind
        for j in train:
            if LB_Keogh(i[:-1],j[:-1],5)<min_dist:
                dist=DTWDistance(i[:-1],j[:-1],w)
                if dist<min_dist:
                    min_dist=dist
                    closest_seq=j
        preds.append(closest_seq[-1])
    prec = precision_score(test[:,-1],preds, average='macro')
    recall= recall_score(test[:,-1],preds, average='macro')
    f1=f1_score(test[:,-1],preds, average='macro')

    return (prec,recall,f1)

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
#knn with Euclidean dist
def knn_euclid(train,test):
    preds=[]
    #index, line value
    for ind,i in enumerate(test):
        min_dist=float('inf')
        closest_seq=[]
        #print ind
        for j in train:
            dist=euclid_dist(i[:-1],j[:-1])
            if dist<min_dist:
                min_dist=dist
                closest_seq=j
        preds.append(closest_seq[-1])
    prec = precision_score(test[:,-1],preds, average='macro')
    recall= recall_score(test[:,-1],preds, average='macro')
    f1=f1_score(test[:,-1],preds, average='macro')

    return (prec,recall,f1)
#classification_report(test[:,-1],preds)

In [35]:
#demo
train = np.genfromtxt('datasets/train.csv', delimiter='\t')
test = np.genfromtxt('datasets/test.csv', delimiter='\t')
s = knn_euclid(train,test)
print s


# knn on our tone data - euclidean and dtw

In [5]:
import random

def split_data(data,test_size):
    test_ind = random.sample(range(len(data)),test_size)
    whole_ind = range(len(data))
    train_ind = [x for x in whole_ind if x not in test_ind]
    test = np.array([data[x] for x in test_ind])
    train = np.array([data[x] for x in train_ind])
    return test, train



from os import listdir
from os.path import isfile, join
mypath="toneData/"
allfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
onlyfiles=[f for f in allfiles if f.endswith('csv')]




In [56]:

    

#analysis:kmeans with euclidean distance with 5 types of representation feature files

score_dict={}
for file in onlyfiles:
    total_score=(0,0,0)
    fileName=mypath+file
    print fileName
    data = np.genfromtxt(fileName, delimiter=',')
    for i in range(num_iter):
        test,train=split_data(data,200)
        scores = knn_euclid(train,test)
        print "scores:",scores
        total_score=addv(total_score,scores)
        print "total score:",total_score
    ave_score=[x/num_iter for x in total_score]
    print ave_score
    score_dict[file]=ave_score
    print "======================================="

toneData/allxudn-3speakers-sorttone.csv
scores: (0.9278979954216906, 0.93482122715993687, 0.92970715055695297)
total score: (0.9278979954216906, 0.93482122715993687, 0.92970715055695297)
scores: (0.94390980776409106, 0.94393996960486326, 0.94378333600631481)
total score: (1.8718078031857817, 1.8787611967648001, 1.8734904865632678)
scores: (0.91754646002357676, 0.92192397751374888, 0.91911243618020022)
total score: (2.7893542632093586, 2.800685174278549, 2.7926029227434679)
scores: (0.95533626707638863, 0.95577321494635592, 0.95532303243878203)
total score: (3.7446905302857472, 3.756458389224905, 3.7479259551822501)
scores: (0.92969907407407404, 0.92966494555164148, 0.92896712812851001)
total score: (4.6743896043598214, 4.6861233347765463, 4.6768930833107598)
scores: (0.93599087433715633, 0.93936778065523907, 0.93369275078666081)
total score: (5.6103804786969782, 5.6254911154317853, 5.6105858340974208)
scores: (0.93876968020643969, 0.9382868937048503, 0.93828370046348619)
total score: (

In [61]:
score_dict['row-yixu-hertz-sorttone.csv']

[0.94936234291128352, 0.94930475041942231, 0.9487375513155818]

In [9]:
num_iter=10

def addv(a,b):
    return tuple(map(sum, zip(a,b)))

score_dict={}
for file in onlyfiles:
    total_score=(0,0,0)
    fileName=mypath+file
    print fileName
    data = np.genfromtxt(fileName, delimiter=',')
    for i in range(num_iter):
        test,train=split_data(data,200)
        scores = knn(train,test,4)
        print "scores:",scores
        total_score=addv(total_score,scores)
        print "total score:",total_score
    ave_score=[x/num_iter for x in total_score]
    print ave_score
    score_dict[file]=ave_score
    print "======================================="

toneData/concatSynth1-sorttone.csv
scores: (0.84358940279992911, 0.84804894179894186, 0.84170135728966022)
total score: (0.84358940279992911, 0.84804894179894186, 0.84170135728966022)
scores: (0.83787371912622821, 0.83851119894598158, 0.83579334157550211)
total score: (1.6814631219261573, 1.6865601407449233, 1.6774946988651624)
scores: (0.78280423280423284, 0.78297529538131039, 0.78209605270614391)
total score: (2.4642673547303904, 2.4695354361262338, 2.4595907515713065)
scores: (0.77547621291837332, 0.77581281349749198, 0.77195873297953499)
total score: (3.2397435676487638, 3.245348249623726, 3.2315494845508415)
scores: (0.81801011488511488, 0.81434741759147999, 0.81333299198963938)
total score: (4.0577536825338782, 4.0596956672152062, 4.0448824765404812)
scores: (0.75295176463558744, 0.75455901856763918, 0.7485710142092693)
total score: (4.8107054471694655, 4.8142546857828457, 4.7934534907497506)
scores: (0.79528966012016855, 0.78715373449150816, 0.78846181346181343)
total score: (5.

# SAX classificaiton knn


In [111]:
a = np.zeros(shape=(5,2))


In [112]:
a

array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.]])

In [116]:
a[3,1]=1.2

In [117]:
a

array([[ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  1.2],
       [ 0. ,  0. ]])

In [13]:
from saxpy import SAX
def min_dist_sax(t1String,t2String,word,alpha,eps=0.000001):
    s=SAX(word,alpha,eps)
    return s.compare_strings(t1String,t2String)


def convert_sax(ts,word,alpha,eps=0.000001):
    s=SAX(word,alpha,eps)
    (t1String, t1Indices) = s.to_letter_rep(ts)
    return t1String

In [14]:
#convert all data to sax
def data_sax(data,word,alpha):
    data_sax=[]
    for ts in data:
        ts_string=convert_sax(ts[:-1],word,alpha)
        ts_string+=str(int(ts[-1]))
        data_sax.append(ts_string)
    return data_sax


In [154]:
a=data_sax(data,word,alpha)

In [155]:
print a[:100]

['aabcddd1', 'aabcddc1', 'abccddd1', 'aabcddc1', 'aabbcdd1', 'aabcddd1', 'aacddcb1', 'aabcddd1', 'aacddcc1', 'aabcddd1', 'aaabcdd1', 'aabcddd1', 'aabccdd1', 'aabcddd1', 'aabcddd1', 'aabbcdd1', 'adddbba1', 'aabccdd1', 'acdddba1', 'aabbcdd1', 'aabccdd1', 'aacddcc1', 'bbcddba1', 'aabcddc1', 'aabcddd1', 'aabcddd1', 'baaacdd1', 'dbbcbaa1', 'aabcddd1', 'aabcddd1', 'dcbbabc1', 'aabcddd1', 'bbbcdda1', 'ccddbaa1', 'aabcddd1', 'aacdddc1', 'aabccdc1', 'aaabddd1', 'abccccd1', 'aabcddd1', 'aacddcc1', 'dbabdca1', 'aabcddd1', 'aabcddd1', 'aabcddd1', 'aabcddd1', 'dddbaaa1', 'aaccddd1', 'aaabddd1', 'aabcddc1', 'aabcddd1', 'aabcddd1', 'aabcddd1', 'aabcddd1', 'abbbcdd1', 'aabcddd1', 'aabcddd1', 'aabcddd1', 'aabcddd1', 'aabcddd1', 'aabcddd1', 'aabcddd1', 'aabcddd1', 'abddccc1', 'aabcddd1', 'abbbcdd1', 'aabcddd1', 'aabcddd1', 'aabbcdd1', 'aabcddd1', 'aabcddd1', 'aabcddd1', 'acddcba1', 'bbddcaa1', 'abddcbb1', 'abbccdd1', 'aacddca1', 'dccdcaa1', 'abcddcc1', 'aabcddd1', 'aaacddd1', 'aacddcc1', 'abccbdd1', 'aa

In [189]:
data[0]

array([ -6.27248860e-02,  -5.98698290e-02,  -5.62488890e-02,
        -5.17472450e-02,  -4.83555100e-02,  -4.37595120e-02,
        -3.71938240e-02,  -2.98960330e-02,  -2.23343880e-02,
        -1.44712130e-02,  -7.18623600e-03,  -2.55626000e-04,
         6.09065600e-03,   1.12446530e-02,   1.56460630e-02,
         1.87748920e-02,   2.13038610e-02,   2.34514510e-02,
         2.51449490e-02,   2.68146640e-02,   2.84535490e-02,
         2.98095120e-02,   3.09127900e-02,   3.17744060e-02,
         3.17543630e-02,   3.12349470e-02,   3.01104630e-02,
         2.75978310e-02,   2.42377950e-02,   1.96863470e-02,
         1.00000000e+00])

In [157]:
len(all_sax)

10

In [158]:
print len(all_sax[2])

1920


In [167]:
min_dist_sax(all_sax[2][0][:-1],all_sax[2][988][:-1],5,3)

1.4895636945092345

In [15]:
def knn_sax(train,test,word,alpha):
    preds=[]
    #index, line value
    for ind,i in enumerate(test):
        min_dist=float('inf')
        closest_seq=[]
        #HeRE i and j are just two strings 'abcd1' and 'abcc2' e.g.
        for j in train:
            dist=min_dist_sax(i[:-1],j[:-1],word,alpha)
            if dist<min_dist:
                min_dist=dist
                closest_seq=j
                #print 'j',j
        preds.append(closest_seq[-1])
    #print 'preds',preds
    gtruth=[x[-1] for x in test]
    prec = precision_score(gtruth,preds, average='macro')
    recall= recall_score(gtruth,preds, average='macro')
    f1=f1_score(gtruth,preds, average='macro')

    return (prec,recall,f1)

In [10]:
#convert all data to sax
def data_sax(data,word,alpha):
    data_sax=[]
    for ts in data:
        ts_string=convert_sax(ts[:-1],word,alpha)
        ts_string+=str(int(ts[-1]))
        data_sax.append(ts_string)
    return data_sax

#try different parameter combinations for a data file (such as D1 data or bk data file)
def generate_sax_data(data):
    all_sax=[]
    all_pars=[]
    for word in range(18,20):
        for alpha in range(6,8):
            a=data_sax(data,word,alpha)
            all_sax.append(a)
            all_pars.append((word,alpha))
    return all_sax,all_pars
        



In [11]:
from os import listdir
from os.path import isfile, join
mypath="toneSub/"
allfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
onlyfiles=[f for f in allfiles if f.endswith('csv')]
all_data=[]
for file in onlyfiles:
    total_score=(0,0,0)
    fileName=mypath+file
    print fileName
    data = np.genfromtxt(fileName, delimiter=',')
    all_data.append(data)

toneSub/row-yixunorm-bk-sorttone.csv


In [ ]:
#there are several data files, stored in all_data. for each data file in all data, you can generate multiple sax files
#by permutating the word and alpha values using the generate_sax_data() function. 


In [16]:
num_iter=1
score_dict_sax={}
for data_file in all_data:
    all_sax,all_pars=generate_sax_data(data_file)
    print 'generated file from ' + str(len(all_data)) + ' data file(s)'
    for i in range(len(all_sax)):
        data=all_sax[i]
        data_pars=all_pars[i]
        total_score=(0,0,0)
        print "sax parameters:",data_pars
        #fileName=mypath+file
        #print fileName
        #data = np.genfromtxt(fileName, delimiter=',')
        for i in range(num_iter):
            best_so_far=(0,0,0)
            test,train=split_data(data,200)
            scores = knn_sax(train,test,data_pars[0],data_pars[1])
            print 'scores:',scores
            if scores[-1]>best_so_far[-1]:
                best_so_far=scores                
            print "scores:",best_so_far
            total_score=addv(total_score,scores)
            print "total score:",total_score
        ave_score=[x/num_iter for x in total_score]
        print ave_score
        #score_dict_sax[file]=ave_score
        print "======================================="

generated file from 1 data file(s)
sax parameters: (18, 6)
scores: (0.29022988505747127, 0.35909090909090907, 0.26785714285714285)
scores: (0.29022988505747127, 0.35909090909090907, 0.26785714285714285)
total score: (0.29022988505747127, 0.35909090909090907, 0.26785714285714285)
[0.29022988505747127, 0.35909090909090907, 0.26785714285714285]
sax parameters: (18, 7)
scores: (0.25996677740863788, 0.36189310485516113, 0.2631158127806138)
scores: (0.25996677740863788, 0.36189310485516113, 0.2631158127806138)
total score: (0.25996677740863788, 0.36189310485516113, 0.2631158127806138)
[0.25996677740863788, 0.36189310485516113, 0.2631158127806138]
sax parameters: (19, 6)
scores: (0.28113553113553114, 0.32870370370370372, 0.23133575464083939)
scores: (0.28113553113553114, 0.32870370370370372, 0.23133575464083939)
total score: (0.28113553113553114, 0.32870370370370372, 0.23133575464083939)
[0.28113553113553114, 0.32870370370370372, 0.23133575464083939]
sax parameters: (19, 7)
scores: (0.5392857

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


generated file from 1 data file(s)
sax parameters: (8, 3)
scores: (0.056250000000000001, 0.25, 0.091836734693877542)
scores: (0.056250000000000001, 0.25, 0.091836734693877542)
total score: (0.056250000000000001, 0.25, 0.091836734693877542)
[0.056250000000000001, 0.25, 0.091836734693877542]
=======================================
sax parameters: (8, 4)
scores: (0.056250000000000001, 0.25, 0.091836734693877542)
scores: (0.056250000000000001, 0.25, 0.091836734693877542)
total score: (0.056250000000000001, 0.25, 0.091836734693877542)
[0.056250000000000001, 0.25, 0.091836734693877542]
=======================================
sax parameters: (8, 5)
scores: (0.22758037225042299, 0.26111111111111113, 0.11879251700680271)
scores: (0.22758037225042299, 0.26111111111111113, 0.11879251700680271)
total score: (0.22758037225042299, 0.26111111111111113, 0.11879251700680271)
[0.22758037225042299, 0.26111111111111113, 0.11879251700680271]
=======================================
sax parameters: (8, 6)
scores: (0.26984588543181154, 0.31206948188080263, 0.22427140255009109)
scores: (0.26984588543181154, 0.31206948188080263, 0.22427140255009109)
total score: (0.26984588543181154, 0.31206948188080263, 0.22427140255009109)
[0.26984588543181154, 0.31206948188080263, 0.22427140255009109]
=======================================
sax parameters: (9, 3)
scores: (0.068750000000000006, 0.25, 0.10784313725490198)
scores: (0.068750000000000006, 0.25, 0.10784313725490198)
total score: (0.068750000000000006, 0.25, 0.10784313725490198)
[0.068750000000000006, 0.25, 0.10784313725490198]
=======================================
sax parameters: (9, 4)
scores: (0.071249999999999994, 0.25, 0.11089494163424124)
scores: (0.071249999999999994, 0.25, 0.11089494163424124)
total score: (0.071249999999999994, 0.25, 0.11089494163424124)
[0.071249999999999994, 0.25, 0.11089494163424124]
=======================================
sax parameters: (9, 5)
scores: (0.28423390081421168, 0.28061224489795916, 0.16288316946211681)
scores: (0.28423390081421168, 0.28061224489795916, 0.16288316946211681)
total score: (0.28423390081421168, 0.28061224489795916, 0.16288316946211681)
[0.28423390081421168, 0.28061224489795916, 0.16288316946211681]
=======================================